In [ ]:

!pip install pytorch-tabnet


In [ ]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
import matplotlib.pyplot as plt


In [ ]:

# Example DataFrame loading - replace with actual data
# df = pd.read_csv('your_data.csv')
# Assuming 'target' is your prediction column

# Sample placeholder
df = pd.DataFrame({
    'feature1': np.random.rand(1000),
    'feature2': np.random.randint(0, 5, size=1000),
    'target': np.random.rand(1000)
})

X = df.drop(columns=["target"])
y = df["target"].values

# Encode categorical features if any
for col in X.select_dtypes("object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

X_train, X_valid, y_train, y_valid = train_test_split(X.values, y, test_size=0.2, random_state=42)


In [ ]:

tabnet = TabNetRegressor(verbose=1)

tabnet.fit(
    X_train=X_train, y_train=y_train.reshape(-1, 1),
    eval_set=[(X_valid, y_valid.reshape(-1, 1))],
    eval_name=["val"],
    eval_metric=["rmse"],
    max_epochs=200,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0
)


In [ ]:

preds = tabnet.predict(X_valid)

# Basic evaluation
from sklearn.metrics import mean_squared_error
print("RMSE:", mean_squared_error(y_valid, preds, squared=False))


In [ ]:

plt.figure(figsize=(10, 6))
plt.barh(df.drop(columns=["target"]).columns, tabnet.feature_importances_)
plt.title("TabNet Feature Importances")
plt.xlabel("Importance")
plt.show()
